In [2]:
import datetime as dt
import thredds_lsasaf_utils as tlu
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.mask import mask
from shapely import wkt

import statsmodels.api as sm
from statsmodels.genmod.families import Gaussian
from statsmodels.genmod.families.links import Power
from statsmodels.genmod.families import Gamma
from statsmodels.genmod.families.links import log, identity

import pickle
import os
import thredds_lsasaf_utils as tlu

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.families import Gaussian
from statsmodels.genmod.families.links import Power
from statsmodels.genmod.families import Gamma
from statsmodels.genmod.families.links import log, identity

import pickle

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import patsy
import rasterio
from shapely.geometry import Polygon, Point
from rasterio.mask import mask

import common_utils

In [1]:
import geopandas as gpd
from shapely.geometry import Point

def get_msg_centroids(msg_lst_sample_csv_file):
    """
    Get the centroid of the 5k grids.

    Parameters:
        msg_lst_sample_csv_file (str): Path to the CSV file containing lon and lat columns.

    Returns:
        GeoDataFrame: A GeoDataFrame containing unique geometries (centroids) with precision up to 6 digits.
    """

    # Step 1: Read the CSV file into a GeoDataFrame
    gdf = common_utils.read_fused_data(msg_lst_sample_csv_file)

    # Step 2: Create geometry Point from lon and lat columns (rounding to 6 decimal places)
    gdf['geometry'] = gdf.apply(
        lambda row: Point(round(row['lon'], 6), round(row['lat'], 6)), axis=1
    )

    # Step 3: Keep only unique Points
    gdf = gdf.drop_duplicates(subset='geometry')

    # Step 4: Retain only the geometry column
    gdf = gdf[['geometry']]

    return gdf


In [5]:
import rasterio
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

def get_eco_stress_lst_centroids(eco_stress_tif_file):
    """
    Get the lat lon of the raster pixels.

    Parameters:
        eco_stress_tif_file (str): Path to the tif file containing lst values.

    Returns:
        GeoDataFrame: A GeoDataFrame containing unique geometries (Points) with precision up to 6 digits.
    """
    # Step 1: Open the raster file using rasterio
    with rasterio.open(eco_stress_tif_file) as src:
        # Step 2: Read the raster data into an array (single band assumed)
        band = src.read(1)

        # Step 3: Get the affine transformation (to map pixel coordinates to geographic coordinates)
        transform = src.transform

        # Step 4: Get pixel coordinates (rows and columns)
        rows, cols = np.where(~np.isnan(band))  # Find non-NaN pixels

        # Step 5: Convert pixel coordinates to geographic coordinates (lat, lon)
        lon, lat = rasterio.transform.xy(transform, rows, cols)

        # Step 6: Create a GeoDataFrame with these points (rounded to 6 decimal places)
        points = [Point(round(lon[i], 6), round(lat[i], 6)) for i in range(len(lon))]
        gdf = gpd.GeoDataFrame(geometry=points, crs="EPSG:4326")  # Assuming WGS84 coordinate reference system

    # Step 7: Remove duplicate points
    gdf = gdf.drop_duplicates(subset='geometry')

    return gdf


In [ ]:
# Assuming the CSV file contains 'lon' and 'lat' columns
file_path = "../downloads/MSG_2ND/LST_2023-6-1_2023-6-1.csv"
centroids_gdf = get_msg_centroids(file_path)

print(centroids_gdf.shape)

# Output file
output_filepath = "../lst_centroids/msg_lst_rome_5k_centroids.gpkg"

# Save the resulting GeoDataFrame
centroids_gdf.to_file(output_filepath, driver="GPKG")

centroids_gdf.head()

In [6]:
eco_stress_lst = "../downloads/ecostress_LST/ECO2LSTE.001_SDS_LST_doy2024206143747_aid0001.tif"

centroids_gdf = get_eco_stress_lst_centroids(eco_stress_lst)

print(centroids_gdf.shape)

# Output file
output_filepath = "../lst_centroids/ecostress_lst_rome_centroids.gpkg"

# Save the resulting GeoDataFrame
centroids_gdf.to_file(output_filepath, driver="GPKG")

centroids_gdf.head()


(372051, 1)


,geometry
0,POINT (12.24959 42.09943)
1,POINT (12.25022 42.09943)
2,POINT (12.25085 42.09943)
3,POINT (12.25148 42.09943)
4,POINT (12.25211 42.09943)
